In [ ]:
!python --version
!pip list | grep tensorflow

In [ ]:
import pandas as pd
import numpy as np
import pytz

In [ ]:
data_src_2018 = "../data/raw/solar_data_2018.csv"
data_df_2018 = pd.read_csv(data_src_2018)
df_headers = data_df_2018.iloc[:1]

In [ ]:
print(df_headers.head())
print(df_headers.columns)

In [ ]:
data_frames = {}

# Load and process data
for year in range(2018, 2024):
    data_src = f"../data/raw/solar_data_{year}.csv"
    data_frames[year] = pd.read_csv(data_src)

    # Process the DataFrame
    data_frames[year] = data_frames[year].iloc[1:]  # Remove first row
    data_frames[year].columns = data_frames[year].iloc[0]  # Set column names
    data_frames[year] = data_frames[year][1:]  # Drop first row after setting headers
    data_frames[year].reset_index(drop=True, inplace=True)  # Reset index


In [ ]:
# Concatenate all DataFrames into one
data_df = pd.concat(data_frames.values(), ignore_index=True)

# Optionally, save to a new CSV file
data_df.to_csv("../data/preprocessed/solar_data_combined.csv", index=False)

In [ ]:
print(data_df.shape)

In [ ]:
#Drop Columns With All Values NaN
df_cleaned = data_df.dropna(axis=1, how="all")
print(df_cleaned.head())

In [ ]:
# Convert all columns to float, replacing non-numeric values with NaN
df_cleaned = df_cleaned.apply(pd.to_numeric, errors='coerce')
print(df_cleaned.dtypes)

In [ ]:
#Join The Date Columns Into A Single Column Called DateTime
df_cleaned['datetime'] = pd.to_datetime(df_cleaned[['Year', 'Month', 'Day', 'Hour', 'Minute']])

In [ ]:
local_time_zone = 'America/Denver'
# Convert UTC to local time
df_cleaned['local_datetime'] = df_cleaned['datetime'].dt.tz_localize('UTC').dt.tz_convert(local_time_zone)
df_cleaned['local_datetime'] = df_cleaned['local_datetime'].dt.tz_localize(None)

In [ ]:
print(df_cleaned)

In [ ]:
# Reorder the columns to make 'datetime' the first column
# Create a list of the columns you want to keep in the specific order
columns_order = ['datetime', 'local_datetime'] + [col for col in df_cleaned.columns if col not in ['datetime', 'local_datetime']]

# Reorder the DataFrame columns based on the list
df_cleaned = df_cleaned[columns_order]

In [ ]:
print(df_cleaned)

In [ ]:
#Testing
print("Number Of NaN Measurements: " + str(df_cleaned.isnull().sum()))

In [ ]:
#Removal Of Rows With Fill Flag != 0
print("Number Of Rows With Fill Flag Error (Before Removal): ", (df_cleaned['Fill Flag'] != 0).sum())
df_cleaned = df_cleaned[df_cleaned["Fill Flag"] == 0]
print("Number Of Rows With Fill Flag Error (After Removal): ", (df_cleaned['Fill Flag'] != 0).sum())

In [ ]:
print(df_cleaned.shape)

In [50]:
#The Cloud Type, Cloud Fill Flag and Fill Flags are dropped
df_cleaned = df_cleaned.drop(columns=['Cloud Type', 'Cloud Fill Flag', 'Fill Flag'])

In [52]:
#Identification Of Outliers
GHI_max = 1000
GHI_min = 0

print("Number Of Outliers (Above Max): " + str((df_cleaned['GHI'] > GHI_max).sum()))
#print(df_cleaned.loc[df_cleaned['GHI'] > GHI_max, 'GHI'])
print("Number Of Outliers (Below Min): " + str((df_cleaned['GHI'] < GHI_min).sum()))
#print(df_cleaned.loc[df_cleaned['GHI'] < GHI_min, 'GHI'])

Number Of Outliers (Above Max): 2116
Number Of Outliers (Below Min): 0


In [53]:
#Removal Of Outliers
df_cleaned = df_cleaned[(df_cleaned["GHI"] < GHI_max)]
print("Number Of Outliers (Above Max): " + str((df_cleaned['GHI'] > GHI_max).sum()))
print("Updated Number Of Rows: ", df_cleaned.shape[0])

Number Of Outliers (Above Max): 0
Updated Number Of Rows:  293707


In [54]:
#Number Of Night Measurements
nb_night_measurements = int(100*df_cleaned[df_cleaned["Clearsky GHI"] < 30].shape[0] / df_cleaned.shape[0])
print("Percentage Of Data As Night Measurements: " + str(nb_night_measurements) + "%")

#Removal Of Night Measurements
print("Clear Sky GHI Minimums And Maximums Before Removal Of Night Measurements", df_cleaned["Clearsky GHI"].min(), df_cleaned["Clearsky GHI"].max())
df_cleaned = df_cleaned[df_cleaned["Clearsky GHI"] > 30]

Percentage Of Data As Night Measurements: 53%
Clear Sky GHI Minimums And Maximums Before Removal Of Night Measurements 0 1064


In [55]:
#Testing
print("Clear Sky GHI Minimums And Maximums After Removal Of Night Measurements", df_cleaned["Clearsky GHI"].min(), df_cleaned["Clearsky GHI"].max())
print("Updated Number Of Rows: ", df_cleaned.shape[0])

Clear Sky GHI Minimums And Maximums After Removal Of Night Measurements 31 1064
Updated Number Of Rows:  135556


In [56]:
#Introduction Of Clear Sky Index
df_cleaned['Kcs'] = df_cleaned['GHI'] / df_cleaned["Clearsky GHI"]
#print(df_cleaned['Kcs'])
df_cleaned.replace([np.inf, -np.inf], 0, inplace = True)
print("Kcs Minimums And Maximums", df_cleaned["Kcs"].min(), df_cleaned["Kcs"].max())
df_cleaned['Kcs'] = df_cleaned['Kcs'].round(3)

Kcs Minimums And Maximums 0.014285714285714285 1.0


In [58]:
df_cleaned.reset_index(drop=True, inplace=True)

In [61]:
df_cleaned = df_cleaned.round(3)

In [59]:
print(df_cleaned)

1                 datetime      local_datetime  Year  Month  Day  Hour  \
0      2018-01-01 16:30:00 2018-01-01 09:30:00  2018      1    1    16   
1      2018-01-01 16:40:00 2018-01-01 09:40:00  2018      1    1    16   
2      2018-01-01 16:50:00 2018-01-01 09:50:00  2018      1    1    16   
3      2018-01-01 17:00:00 2018-01-01 10:00:00  2018      1    1    17   
4      2018-01-01 17:10:00 2018-01-01 10:10:00  2018      1    1    17   
...                    ...                 ...   ...    ...  ...   ...   
135551 2023-12-31 22:00:00 2023-12-31 15:00:00  2023     12   31    22   
135552 2023-12-31 22:10:00 2023-12-31 15:10:00  2023     12   31    22   
135553 2023-12-31 22:20:00 2023-12-31 15:20:00  2023     12   31    22   
135554 2023-12-31 22:30:00 2023-12-31 15:30:00  2023     12   31    22   
135555 2023-12-31 22:40:00 2023-12-31 15:40:00  2023     12   31    22   

1       Minute  Temperature  Alpha    AOD  ...  Ozone  Relative Humidity  \
0           30         -1.2   1.20 

In [62]:
df_cleaned.to_csv("../data/preprocessed/solar_data_combined_cleaned.csv", index=False)